In [3]:
import pandas as pd
import pymysql  # Make sure to import the driver
from sqlalchemy import create_engine, text

In [68]:
# Database credentials
username = "root"
password = "abdul256"
host = "localhost"
port = "3306"
database = "sqlbasic"

# Create the engine with pymysql
engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")

# helper functin select
def select(query):
    return pd.read_sql(query,  con = engine)


In [5]:
# Loading our data
purchases_behaviour = pd.read_csv("../SQL/datasets/QVI_purchase_behaviour.csv")
transactions_data = pd.read_csv("../SQL/datasets/QVI_transaction_data.csv")

In [4]:
purchases_behaviour.head()

,LYLTY_CARD_NBR,LIFESTAGE,PREMIUM_CUSTOMER
0,1000,YOUNG SINGLES/COUPLES,Premium
1,1002,YOUNG SINGLES/COUPLES,Mainstream
2,1003,YOUNG FAMILIES,Budget
3,1004,OLDER SINGLES/COUPLES,Mainstream
4,1005,MIDAGE SINGLES/COUPLES,Mainstream


In [5]:
transactions_data.head()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
0,43390,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0
1,43599,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3
2,43605,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9
3,43329,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0
4,43330,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8


In [12]:
purchases_behaviour.isnull().sum()

LYLTY_CARD_NBR      0
LIFESTAGE           0
PREMIUM_CUSTOMER    0
dtype: int64

In [13]:
# making sure we do not have duplicates
purchases_behaviour['LYLTY_CARD_NBR'].value_counts()[lambda x: x > 1]

Series([], Name: count, dtype: int64)

Customer DIM table Overview

In [55]:
customer_dim = purchases_behaviour

In [56]:
customer_dim = customer_dim.rename({'LYLTY_CARD_NBR':'Customer_id','LIFESTAGE':'Category','PREMIUM_CUSTOMER':'Customer_Segment'}, axis=1)

In [57]:
customer_dim

,Customer_id,Category,Customer_Segment
0,1000,YOUNG SINGLES/COUPLES,Premium
1,1002,YOUNG SINGLES/COUPLES,Mainstream
2,1003,YOUNG FAMILIES,Budget
3,1004,OLDER SINGLES/COUPLES,Mainstream
4,1005,MIDAGE SINGLES/COUPLES,Mainstream
...,...,...,...
72632,2370651,MIDAGE SINGLES/COUPLES,Mainstream
72633,2370701,YOUNG FAMILIES,Mainstream
72634,2370751,YOUNG FAMILIES,Premium
72635,2370961,OLDER FAMILIES,Budget


Products DIM table

In [36]:
product_dim = transactions_data[['PROD_NBR','PROD_NAME']]

Transactions Table

In [96]:
transaction_fact = transactions_data[['LYLTY_CARD_NBR',
                                      'TXN_ID',
                                      'PROD_NBR',
                                      'PROD_QTY',
                                      'TOT_SALES',
                                      'DATE']].rename({'LYLTY_CARD_NBR':'Customer_id',
                                                       'TXN_ID':'Transaction_id',
                                                       'PROD_NBR':'Product_id',
                                                       'PROD_QTY':'Product_QTY',
                                                       'TOT_SALES':'Total_Sales',
                                                       'DATE':'Date'}, axis = 1)

In [60]:
customer_dim.head(4)

,Customer_id,Category,Customer_Segment
0,1000,YOUNG SINGLES/COUPLES,Premium
1,1002,YOUNG SINGLES/COUPLES,Mainstream
2,1003,YOUNG FAMILIES,Budget
3,1004,OLDER SINGLES/COUPLES,Mainstream


In [27]:
product_dim.head(4)

,PROD_NBR,PROD_NAME
0,5,Natural Chip Compny SeaSalt175g
1,66,CCs Nacho Cheese 175g
2,61,Smiths Crinkle Cut Chips Chicken 170g
3,69,Smiths Chip Thinly S/Cream&Onion 175g


In [66]:
transaction_fact.head(4)

,Customer_id,Transaction_id,Product_id,Product_QTY,Total_Sales,Date
0,1000,1,5,2,6.0,43390
1,1307,348,66,3,6.3,43599
2,1343,383,61,2,2.9,43605
3,2373,974,69,5,15.0,43329


In [ ]:
customer_dim.to_sql(name='Customer_Dim', con=engine, if_exists='replace', index=False)
product_dim.to_sql(name='Product_Dim', con=engine, if_exists='replace', index=False)
transaction_fact.to_sql(name='Transactions_Fact', con=engine, if_exists='replace', index=False)

In [ ]:
purchases_behaviour.to_sql(name='Purchases_Behaviour', con=engine, if_exists='replace', index=False)
transactions_data.to_sql(name='Transactions_Data', con=engine, if_exists='replace', index=False)

In [80]:
query = '''SELECT LYLTY_CARD_NBR AS Customer_id, 
                  LIFESTAGE AS Category, 
                  PREMIUM_CUSTOMER AS Customer_Segment 
           FROM Purchases_Behaviour'''
select(query)

,Customer_id,Category,Customer_Segment
0,1000,YOUNG SINGLES/COUPLES,Premium
1,1002,YOUNG SINGLES/COUPLES,Mainstream
2,1003,YOUNG FAMILIES,Budget
3,1004,OLDER SINGLES/COUPLES,Mainstream
4,1005,MIDAGE SINGLES/COUPLES,Mainstream
...,...,...,...
72632,2370651,MIDAGE SINGLES/COUPLES,Mainstream
72633,2370701,YOUNG FAMILIES,Mainstream
72634,2370751,YOUNG FAMILIES,Premium
72635,2370961,OLDER FAMILIES,Budget


In [247]:
query = '''SELECT PROD_NBR AS Product_id, PROD_NAME AS Product_Name FROM Transactions_Data'''
select(query)

,Product_id,Product_Name
0,5,5
1,66,66
2,61,61
3,69,69
4,108,108
...,...,...
264831,89,89
264832,74,74
264833,51,51
264834,42,42


In [220]:
product_weight2 = product_dim['PROD_NAME'].str[-4:].unique()

In [223]:
product_weight2

array(['175g', '170g', '150g', '300g', '330g', '210g', '210G', '270g',
       '220g', '125g', '110g', '134g', '150G', '175G', '380g', '180g',
       '165g', 'Salt', '250g', '200g', '160g', '190g', ' 90g', ' 70g'],
      dtype=object)

In [124]:
product_dim[product_dim['PROD_NAME'].str[-4:] == 'Salt']

,PROD_NBR,PROD_NAME
65,63,Kettle 135g Swt Pot Sea Salt
153,63,Kettle 135g Swt Pot Sea Salt
174,63,Kettle 135g Swt Pot Sea Salt
177,63,Kettle 135g Swt Pot Sea Salt
348,63,Kettle 135g Swt Pot Sea Salt
...,...,...
264564,63,Kettle 135g Swt Pot Sea Salt
264574,63,Kettle 135g Swt Pot Sea Salt
264725,63,Kettle 135g Swt Pot Sea Salt
264767,63,Kettle 135g Swt Pot Sea Salt


In [219]:
product_weights = product_dim['PROD_NAME'].str.split(' ').str[0].unique()

In [203]:
product_dim[product_dim['PROD_NAME'].str.split(' ').str[0] == 'Old']

,PROD_NBR,PROD_NAME
5,57,Old El Paso Salsa Dip Tomato Mild 300g
25,57,Old El Paso Salsa Dip Tomato Mild 300g
71,65,Old El Paso Salsa Dip Chnky Tom Ht300g
87,59,Old El Paso Salsa Dip Tomato Med 300g
114,59,Old El Paso Salsa Dip Tomato Med 300g
...,...,...
264594,65,Old El Paso Salsa Dip Chnky Tom Ht300g
264607,65,Old El Paso Salsa Dip Chnky Tom Ht300g
264615,57,Old El Paso Salsa Dip Tomato Mild 300g
264628,65,Old El Paso Salsa Dip Chnky Tom Ht300g


In [204]:
transactions_data['PACK_SIZE'] = transactions_data['PROD_NAME'].str.extract('(\d+)')

In [207]:
transactions_data['PACK_SIZE'].unique()

array(['175', '170', '150', '300', '330', '210', '270', '220', '125',
       '110', '134', '380', '180', '165', '135', '250', '200', '160',
       '190', '90', '70'], dtype=object)

In [216]:
product_weight['size'] = product_dim['PROD_NAME'].str.extract('(\d+)')

In [225]:
product_weight['size'].unique()

array(['175', '170', '150', '300', '330', '210', '270', '220', '125',
       '110', '134', '380', '180', '165', '135', '250', '200', '160',
       '190', '90', '70'], dtype=object)

In [243]:
query = '''SELECT DISTINCT PROD_NAME, 
                  REGEXP_SUBSTR(PROD_NAME, '[0-9]+') AS size 
           FROM Transactions_Data
           WHERE LOWER(PROD_NAME) LIKE '%chip%'
           OR LOWER(PROD_NAME) LIKE '%crisps%'
           OR LOWER(PROD_NAME) LIKE '%doritos%'
           OR LOWER(PROD_NAME) LIKE '%thins%';
        '''

select(query)

,PROD_NAME,size
0,Old El Paso Salsa Dip Tomato Mild 300g,300
1,Smiths Crinkle Chips Salt & Vinegar 330g,330
2,Smiths Crinkle Original 330g,330
3,Dorito Corn Chp Supreme 380g,380
4,Doritos Salsa Medium 300g,300
5,Old El Paso Salsa Dip Chnky Tom Ht300g,300
6,Woolworths Mild Salsa 300g,300
7,Old El Paso Salsa Dip Tomato Med 300g,300
8,Smiths Crnkle Chip Orgnl Big Bag 380g,380
9,Cheezels Cheese 330g,330


**DIP_DIM**

In [82]:
query = '''SELECT DISTINCT REGEXP_REPLACE(PROD_NAME, '[0-9]+g', '') AS product_name,
                  SUBSTRING_INDEX(PROD_NAME, ' ', 3) as brand_name,
                  REGEXP_SUBSTR(PROD_NAME, '[0-9]+') AS package_size 
            FROM Transactions_Data
           
           WHERE LOWER(PROD_NAME) LIKE '%%sauce%%'
              OR LOWER(PROD_NAME) LIKE '%%dip%%'
              OR LOWER(PROD_NAME) LIKE '%%salsa%%'
              OR LOWER(PROD_NAME) LIKE '%%mild%%'
              

            '''

select(query)

,product_name,brand_name,package_size
0,Old El Paso Salsa Dip Tomato Mild,Old El Paso,300
1,Red Rock Deli SR Salsa & Mzzrlla,Red Rock Deli,150
2,Smiths Crinkle Cut French OnionDip,Smiths Crinkle Cut,150
3,Smiths Crinkle Cut Tomato Salsa,Smiths Crinkle Cut,150
4,Doritos Salsa Medium,Doritos Salsa,300
5,Old El Paso Salsa Dip Chnky Tom Ht,Old El Paso,300
6,Woolworths Mild Salsa,Woolworths Mild,300
7,Old El Paso Salsa Dip Tomato Med,Old El Paso,300
8,Woolworths Medium Salsa,Woolworths Medium,300
9,Smiths Crinkle Cut Snag&Sauce,Smiths Crinkle Cut,150


**Customer_DIM**

In [90]:
query = '''SELECT LYLTY_CARD_NBR AS Customer_id, 
                  LIFESTAGE AS Category, 
                  PREMIUM_CUSTOMER AS Customer_Segment 
           FROM Purchases_Behaviour'''

select(query)

,Customer_id,Category,Customer_Segment
0,1000,YOUNG SINGLES/COUPLES,Premium
1,1002,YOUNG SINGLES/COUPLES,Mainstream
2,1003,YOUNG FAMILIES,Budget
3,1004,OLDER SINGLES/COUPLES,Mainstream
4,1005,MIDAGE SINGLES/COUPLES,Mainstream
...,...,...,...
72632,2370651,MIDAGE SINGLES/COUPLES,Mainstream
72633,2370701,YOUNG FAMILIES,Mainstream
72634,2370751,YOUNG FAMILIES,Premium
72635,2370961,OLDER FAMILIES,Budget


In [73]:
query = '''SELECT * FROM brands_map'''
select(query)

,brand_name,clean_brand
0,Natural,Natural Chip
1,CCs,CCs
2,Smiths,Smiths
3,Kettle,Kettle
4,Old,Old
5,Grain,Sunbites
6,Doritos,Doritos
7,Twisties,Twisties
8,WW,WW
9,Thins,Thins


**CHIPS_DIM**

In [89]:
query = '''WITH chips_raw AS (SELECT DISTINCT REGEXP_REPLACE(PROD_NAME, '[0-9]+g', '') AS product_name, PROD_NBR,
           SUBSTRING_INDEX(PROD_NAME, ' ', 1) as brand_name,
           REGEXP_SUBSTR(PROD_NAME, '[0-9]+') AS package_size 
           FROM Transactions_Data
           WHERE NOT LOWER(PROD_NAME) LIKE '%%sauce%%'
              OR NOT LOWER(PROD_NAME) LIKE '%%dip%%'
              OR NOT LOWER(PROD_NAME) LIKE '%%salsa%%'
              OR NOT LOWER(PROD_NAME) LIKE '%%mild%%')
            
            SELECT dr.PROD_NBR, dr.product_name,  m.clean_brand AS product_brand, dr.package_size FROM chips_raw dr
            LEFT JOIN brands_map m ON m.brand_name = dr.brand_name
           '''

select(query)

,PROD_NBR,product_name,product_brand,package_size
0,5,Natural Chip Compny SeaSalt,Natural Chip,175
1,66,CCs Nacho Cheese,CCs,175
2,61,Smiths Crinkle Cut Chips Chicken,Smiths,170
3,69,Smiths Chip Thinly S/Cream&Onion,Smiths,175
4,108,Kettle Tortilla ChpsHny&Jlpno Chili,Kettle,150
...,...,...,...,...
109,58,Red Rock Deli Chikn&Garlic Aioli,Red Rock Deli,150
110,10,RRD SR Slow Rst Pork Belly,Red Rock Deli,150
111,11,RRD Pc Sea Salt,Red Rock Deli,165
112,43,Smith Crinkle Cut Bolognese,Smiths,150


In [77]:
brands = pd.DataFrame()
brands['brand_name'] = transactions_data['PROD_NAME'].str.split().str[0].unique()

clean_brands = {
    'Natural': 'Natural Chip',
    'Dorito': 'Doritos',
    'Infzns': 'Infuzions',
    'Snbts': 'Sunbites',
    'Smith': 'Smiths',
    'Burger': 'Smiths',
    'Grain':'Sunbites',
    'NCC':'Natural Chip',
    'GrnWves':'Sunbites',
    'Red':'Red Rock Deli',
    'RRD':'Red Rock Deli',
    'French':'French Fries',
    
}

brands['clean_brand'] = brands['brand_name'].replace(clean_brands)
brands

,brand_name,clean_brand
0,Natural,Natural Chip
1,CCs,CCs
2,Smiths,Smiths
3,Kettle,Kettle
4,Old,Old
5,Grain,Sunbites
6,Doritos,Doritos
7,Twisties,Twisties
8,WW,WW
9,Thins,Thins


**Transaction_FACT**

In [103]:
query = '''SELECT TXN_ID AS Transaction_ID, 
                  LYLTY_CARD_NBR AS Customer_ID, 
                  PROD_NBR AS Product_ID, 
                  PROD_QTY AS Product_Quantity, 
                  TOT_SALES AS Total_Sales,
                  DATE_ADD('1899-12-30', INTERVAL DATE DAY) AS Date
            FROM Transactions_Data'''

select(query)

,Transaction_ID,Customer_ID,Product_ID,Product_Quantity,Total_Sales,Date
0,1,1000,5,2,6.0,2018-10-17
1,348,1307,66,3,6.3,2019-05-14
2,383,1343,61,2,2.9,2019-05-20
3,974,2373,69,5,15.0,2018-08-17
4,1038,2426,108,3,13.8,2018-08-18
...,...,...,...,...,...,...
264831,270088,272319,89,2,10.8,2019-03-09
264832,270154,272358,74,1,4.4,2018-08-13
264833,270187,272379,51,2,8.8,2018-11-06
264834,270188,272379,42,2,7.8,2018-12-27


In [104]:
transaction_fact.dtypes

Customer_id         int64
Transaction_id      int64
Product_id          int64
Product_QTY         int64
Total_Sales       float64
Date                int64
dtype: object

In [40]:
transactions_data[transactions_data['PROD_NAME'].str.split().str[0] == 'French']

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
86,43604,115,115381,118981,29,French Fries Potato Chips 175g,1,3.0
680,43363,33,33019,29543,29,French Fries Potato Chips 175g,2,6.0
780,43337,39,39134,35434,29,French Fries Potato Chips 175g,2,6.0
1299,43450,69,69113,66543,29,French Fries Potato Chips 175g,2,6.0
1379,43362,73,73390,72741,29,French Fries Potato Chips 175g,2,6.0
...,...,...,...,...,...,...,...,...
263812,43500,229,229158,231300,29,French Fries Potato Chips 175g,2,6.0
264517,43469,258,258348,257377,29,French Fries Potato Chips 175g,2,6.0
264557,43498,259,259226,258909,29,French Fries Potato Chips 175g,2,6.0
264667,43645,265,265032,263333,29,French Fries Potato Chips 175g,1,3.0


In [196]:
query = '''SELECT DISTINCT SUBSTRING_INDEX(PROD_NAME, ' ', 1) AS brand FROM Transactions_Data'''

select(query)

,brand
0,Natural
1,CCs
2,Smiths
3,Kettle
4,Old
5,Grain
6,Doritos
7,Twisties
8,WW
9,Thins
